In [7]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [49]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")
SPXT_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPXT")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()
SPXT_Idx = SPXT_Idx.set_index("Date").sort_index()

portfolio.loc[:,"SPYCumDiv"]=portfolio.loc[:,"SPY Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueSPY"]=portfolio.loc[:,"SPY Position"]+portfolio.loc[:,"SPYCumDiv"]
portfolio = portfolio.sort_index()

SPYCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueSPY"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData.loc[:,"TotalReturnsSPY"] = SPYCorrData["TotalValueSPY"].pct_change()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"HY_Index_OAS_Diff"]=SPYCorrData["HY_Index_OAS"].diff()
SPYCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=SPYCorrData["HY_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
SPYCorrData.loc[:,"10yYieldDiff"] = SPYCorrData["10y_UST_INDEX_PX"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
SPYCorrData.loc[:,"IG_Index_OAS_Diff"]=SPYCorrData["IG_Index_OAS"].diff()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=SPYCorrData["IG_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"SPY_Idx_LAST_PX"})
SPYCorrData.loc[:,"ReturnSPY_Idx"] = SPYCorrData["SPY_Idx_LAST_PX"].pct_change()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(SPXT_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"SPXT_Idx_LAST_PX"})
SPYCorrData.loc[:,"ReturnSPXT_Idx"] = SPYCorrData["SPXT_Idx_LAST_PX"].pct_change()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"Vix Idx"})

SPYCorrData=SPYCorrData.dropna()
SPYCorrData = SPYCorrData.sort_index(ascending=False)

In [51]:
SPYCorrData[["TotalReturnsSPY","ReturnSPXT_Idx"]]

,TotalReturnsSPY,ReturnSPXT_Idx
Date,,
2025-12-31,-0.000372,-0.007280
2025-12-30,-0.003036,-0.001374
2025-12-29,0.005725,-0.003461
2025-12-26,0.006404,-0.000209
2025-12-24,0.001795,0.003224
...,...,...
2021-01-19,-0.001495,0.008175
2021-01-15,0.004009,-0.007177
2021-01-14,-0.003474,-0.003638


In [52]:
SPYCorrData[["TotalReturnsSPY","ReturnSPXT_Idx"]].corr()

,TotalReturnsSPY,ReturnSPXT_Idx
TotalReturnsSPY,1.00000,-0.00711
ReturnSPXT_Idx,-0.00711,1.00000


In [50]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(SPYCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPXT_Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPXT_Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

SPYCorrData.loc[:,"PC1"]=X_pca[:,0]
SPYCorrData.loc[:,"PC2"]=X_pca[:,1]
SPYCorrData.loc[:,"PC3"]=X_pca[:,2]
SPYCorrData.loc[:,"PC4"]=X_pca[:,3]
SPYCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(SPYCorrData[["PC1","PC2","PC3","PC4"]])
Y = SPYCorrData["TotalReturnsSPY"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.47302197 0.22356364 0.17043052 0.0866792  0.04630466]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Diff  ReturnSPXT_Idx  \
0           0.587849     -0.163308           0.571732       -0.493780   
1          -0.143810      0.809732          -0.072592       -0.288442   
2          -0.099539     -0.409297          -0.078211        0.332736   
3           0.182210      0.362247           0.556809        0.724004   
4           0.768534      0.137537          -0.593042        0.195159   

    Vix Idx  
0  0.238889  
1  0.484958  
2  0.840082  
3 -0.036842  
4  0.025561  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsSPY   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4962
Date:                Thu, 15 Jan 2026   Prob (F-statistic):              0.739
Time:                        20:03:59   Log-Likelihood:                 3885.8
No. Observations:                1240   AIC:                            -7762.
Df Residuals:                    1235   BIC:                            -7736.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.000      1.908      0.057   -1.61e-05       0.001
PC1         2.567e-05      0.000      0.132      0.895      -0.000       0.000
PC2        -1.878e-05      0.000     -0.066      0.947      -0.001       0.001
PC3            0.0003      0.000      0.977      0.329      -0.000       0.001
PC4           -0.0005      0.000     -1.004      0.315      -0.001       0.000
==============================================================================
Omnibus:                      213.410   Durbin-Watson:                   2.074
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3491.837
Skew:                           0.229   Prob(JB):                         0.00
Kurtosis:                      11.208   Cond. No.                         2.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""